In [1]:
## Open parser.py file and change the code for from parser import Parser to from .parser import Parser
# .conda\envs\untl_py3\Lib\site-packages\vcf

#import vcf

In [2]:
#"S:\\UNTLSynology\Quant\Data\gnomAd"

#vcf.Reader(open(":\\UNTLSynology\Quant\Data\gnomAd", 'r'))

In [3]:
def extract_variants_by_genes(vcf_file, output_file, genes_of_interest):
    """
    Extract variants from a VCF file based on specified gene names.
    
    Parameters:
    - vcf_file: Input VCF file path.
    - output_file: Output VCF file path.
    - genes_of_interest: List of gene names you want to extract variants for.
    """
    with open(output_file, 'w') as out_vcf:
        vcf_reader = vcf.Reader(open(vcf_file, 'r'))
        
        # Write VCF header to the output file
        vcf.Writer(out_vcf, vcf_reader)
        
        # Iterate through each variant in the input VCF file
        for record in vcf_reader:
            # Check if the variant has gene information in the INFO field
            if 'GENE' in record.INFO:
                # Extract the gene names for the variant
                gene_names = record.INFO['GENE'].split(',')
                
                # Check if any of the gene names match the genes of interest
                if any(gene in genes_of_interest for gene in gene_names):
                    # Write the variant to the output VCF file
                    out_vcf.write_record(record)

# Example usage
input_vcf_file = 'your_input.vcf'
output_vcf_file = 'output_filtered.vcf'
genes_of_interest = ['GeneA', 'GeneB']

#extract_variants_by_genes(input_vcf_file, output_vcf_file, genes_of_interest)

In [4]:
import gzip
import pandas as pd

row = ""

def extract_variants_by_gene(vcf_bgz_file, gene_name):
    """
    Extract variants from a bgzipped VCF file based on a specified gene name.
    
    Parameters:
    - vcf_bgz_file: Input bgzipped VCF file path.
    - gene_name: Gene name to filter variants.
    
    Returns:
    - Pandas DataFrame containing variants for the specified gene.
    """
    global row
    variants = []
    i = 0
    with gzip.open(vcf_bgz_file, 'rt') as vcf_file:
        for line in vcf_file:
            if line.startswith('#'):
                continue
            row = line
            i += 1
            #if i > 100: break
            fields = line.strip().split('\t')
            if (int(fields[1]) >= 44943683) and (int(fields[1]) <= 44943837):
                
                print(fields[5])
                
                break
            # Extract gene names from the INFO field
            info_field = fields[7]
            continue
            if gene_name in gene_names:
                variants.append({
                    'Chromosome': fields[0],
                    'Position': int(fields[1]),
                    'rsIDs': fields[2],
                    'ALT': fields[4],
                    'QUAL': fields[5],
                    'FILTER': fields[6],
                    'INFO': info_field
                })
    return pd.DataFrame(variants)

def save_variants_to_file(variants_df, output_file):
    """
    Save variants DataFrame to a CSV or Excel file.
    
    Parameters:
    - variants_df: Pandas DataFrame containing variants.
    - output_file: Output file path.
    """
    if output_file.endswith('.csv'):
        variants_df.to_csv(output_file, index=False)
    elif output_file.endswith('.xlsx'):
        variants_df.to_excel(output_file, index=False)
    else:
        raise ValueError("Output file format not supported. Please use either CSV or Excel.")

# Example usage

input_bgz_vcf_file = 'C:\\Users\\Administrator\\Downloads\\gnomad.genomes.v4.0.sites.chr21.vcf.bgz'
gene_name = 'GeneA'
output_file = 'variants_for_geneA.xlsx'

#variants_df = extract_variants_by_gene(input_bgz_vcf_file, gene_name)
#save_variants_to_file(variants_df, output_file)


In [ ]:
import pandas as pd

vcf_bgz_file = 'C:\\Users\\Administrator\\Downloads\\gnomad.genomes.v4.0.sites.chr21.vcf.bgz'

gtex_df = pd.read_csv("ALTssDB_5ss.csv")

gtex_df.head(10)

gtex_df = gtex_df[gtex_df['chromo'] == 'chr21'].copy()

gtex_df = gtex_df.sort_values("start")

# List to store merged rows
merged_rows = []

i = 0

row1 = gtex_df.iloc[i]

genename = row1['genename']
       
start, end = row1['start'], row1['end']

#start, end = 44943683, 44943837

    
with gzip.open(vcf_bgz_file, 'rt') as vcf_file:
    
    print("start first genename", genename, start, end)
            
    for line in vcf_file:

        if line.startswith('#'):
            continue

        a = line.find("\t", 0 + 1)

        b = line.find("\t", a + 1)

        if a >= b: continue

        Chromosome, Position = line[:a], line[a+1: b]
        
        if int(Position) > gtex_df.iloc[len(gtex_df)-1]['end']:
            
            print("Finish the searching")
            
            break

        if int(Position) > end:

            print("No matching anymore", genename, start, end, "current position", Position)
            
            i = i + 1
            
            row1 = gtex_df.iloc[i]

            genename = row1['genename']

            start, end = row1['start'], row1['end']

            print("start next genename", genename, start, end)


        if (int(Position) >= start) and (int(Position) <= end):

            print("Find one matching", genename, start, end, "current position", Position)

            combined_row = row1.to_dict()

            fields = line.strip().split('\t')


            combined_row.update({"Chromosome": fields[0]})

            combined_row.update({"Position": fields[1]})

            combined_row.update({"rsIDs": fields[2]})

            combined_row.update({"Reference": fields[3]})

            combined_row.update({"Alternate": fields[4]})

            last_field = fields[-1]

            a = last_field.find("AC=")

            a += len("AC=")

            b = last_field[a:].find(";")

            combined_row.update({"Allele Count": last_field[a:a+b]})

            a = last_field.find("AN=")

            a += len("AN=")

            b = last_field[a:].find(";")

            combined_row.update({"Allele Number": last_field[a:a+b]})

            a = last_field.find("AF=")

            a += len("AF=")

            b = last_field[a:].find(";")

            combined_row.update({"Allele Frequency": last_field[a:a+b]})



            a = last_field.find("AC_ami=")

            a += (len("AC_ami="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Count Amish": last_field[a:a+b]})

            a = last_field.find("AN_ami=")

            a += (len("AN_ami="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Number Amish": last_field[a:a+b]})

            a = last_field.find("AF_ami=")

            a += (len("AF_ami="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Frequency Amish": last_field[a:a+b]})


            a = last_field.find("AC_amr=")

            a += (len("AC_amr="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Count Admixed American": last_field[a:a+b]})

            a = last_field.find("AN_amr=")

            a += (len("AN_amr="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Number Admixed American": last_field[a:a+b]})

            a = last_field.find("AF_amr=")

            a += (len("AF_amr="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Frequency Admixed American": last_field[a:a+b]})




            a = last_field.find("AC_asj=")

            a += (len("AC_asj="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Count Ashkenazi Jewish": last_field[a:a+b]})

            a = last_field.find("AN_asj=")

            a += (len("AN_asj="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Number Ashkenazi Jewish": last_field[a:a+b]})

            a = last_field.find("AF_asj=")

            a += (len("AF_asj="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Frequency Ashkenazi Jewish": last_field[a:a+b]})



            a = last_field.find("AC_eas=")

            a += (len("AC_eas="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Count East Asian": last_field[a:a+b]})

            a = last_field.find("AN_eas=")

            a += (len("AN_eas="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Number East Asian": last_field[a:a+b]})

            a = last_field.find("AF_eas=")

            a += (len("AF_eas="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Frequency East Asian": last_field[a:a+b]})


            a = last_field.find("AC_fin=")

            a += (len("AC_fin="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Count European (Finnish)": last_field[a:a+b]})

            a = last_field.find("AN_fin=")

            a += (len("AN_fin="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Number European (Finnish)": last_field[a:a+b]})

            a = last_field.find("AF_fin=")

            a += (len("AF_fin="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Frequency European (Finnish)": last_field[a:a+b]})


            a = last_field.find("AC_nfe=")

            a += (len("AC_nfe="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Count European (non-Finnish)": last_field[a:a+b]})

            a = last_field.find("AN_nfe=")

            a += (len("AN_nfe="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Number European (non-Finnish)": last_field[a:a+b]})

            a = last_field.find("AF_nfe=")

            a += (len("AF_nfe="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Frequency European (non-Finnish)": last_field[a:a+b]})


            a = last_field.find("AC_sas=")

            a += (len("AC_sas="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Count South Asian": last_field[a:a+b]})

            a = last_field.find("AN_sas=")

            a += (len("AN_sas="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Number South Asian": last_field[a:a+b]})

            a = last_field.find("AF_sas=")

            a += (len("AF_sas="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Frequency South Asian": last_field[a:a+b]})


            a = last_field.find("AC_remaining=")

            a += (len("AC_remaining="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Count Remaining": last_field[a:a+b]})

            a = last_field.find("AN_remaining=")

            a += (len("AN_remaining="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Number Remaining": last_field[a:a+b]})

            a = last_field.find("AF_remaining=")

            a += (len("AF_remaining="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Frequency Remaining": last_field[a:a+b]})


            a = last_field.find("AC_afr=")

            a += (len("AC_afr="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Count African/African American": last_field[a:a+b]})

            a = last_field.find("AN_afr=")

            a += (len("AN_afr="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Number African/African American": last_field[a:a+b]})

            a = last_field.find("AF_afr=")

            a += (len("AF_afr="))

            b = last_field[a:].find(";")

            combined_row.update({"Allele Frequency African/African American": last_field[a:a+b]})

            merged_rows.append(combined_row)

            #break

    #break
    

# Convert merged rows to DataFrame
merged_df = pd.DataFrame(merged_rows)

merged_df

start first genename CH507-9B2.3,CH507-9B2.4 5124755 5124875
Find one matching CH507-9B2.3,CH507-9B2.4 5124755 5124875 current position 5124863
No matching anymore CH507-9B2.3,CH507-9B2.4 5124755 5124875 current position 5125038
start next genename LIPI 14163418 14163523
Find one matching LIPI 14163418 14163523 current position 14163418
Find one matching LIPI 14163418 14163523 current position 14163430
Find one matching LIPI 14163418 14163523 current position 14163435
Find one matching LIPI 14163418 14163523 current position 14163436
Find one matching LIPI 14163418 14163523 current position 14163440
Find one matching LIPI 14163418 14163523 current position 14163448
Find one matching LIPI 14163418 14163523 current position 14163453
Find one matching LIPI 14163418 14163523 current position 14163454
Find one matching LIPI 14163418 14163523 current position 14163455
Find one matching LIPI 14163418 14163523 current position 14163457
Find one matching LIPI 14163418 14163523 current position 

In [8]:
gtex_df.iloc[len(gtex_df)-1]['end']

46538602

In [7]:
gtex_df.iloc[0]

chromo                                           chr21
strand                                               -
start                                          5124755
end                                            5124875
genename                       CH507-9B2.3,CH507-9B2.4
Major_Exon_Size                                    120
count                                              202
alt5                                                 7
constitLevel                                     0.967
alt5usageLevel                                   0.033
alt5singleCount                                      7
alt5singleLoc                                  5124674
Alt_Exon_Size                                      201
5.ss.Con                         chr21:5124749-5124758
5.ss.Seq                                     CCTGTGCGT
MaxENT                                            4.27
5.ss.Alt                         chr21:5124668-5124677
5.ss.Alt.Seq                                 GTGGTCAGT
ALTMaxENT 

In [36]:
merged_df

,chromo,strand,start,end,genename,Major_Exon_Size,count,alt5,constitLevel,alt5usageLevel,...,?MaxEnt,Upstream_intronSize,Downstream_intronSize,Upperstream_intron_gcpct,Exon_gcpct,Downstream_intron_gcpct,Alt5Exon_gcpct,Alt5Downstream_intron_gcpct,Chromosome,Position
0,chr21,-,5124755,5124875,"CH507-9B2.3,CH507-9B2.4",120,202,7,0.967,0.033,...,-0.58,149,2952,0.591241,0.53913,0.54,0.545918,0.58,chr21,44943686


In [39]:
a = row.find("\t", 0 + 1)

b = row.find("\t", a + 1)

row[:a], row[a+1: b]

row

'chr21\t44943686\trs535219131\tC\tA\t.\tPASS\tAC=4;AN=152232;AF=2.62757e-05;grpmax=amr;fafmax_faf95_max=8.87700e-05;fafmax_faf95_max_gen_anc=amr;AC_XX=0;AF_XX=0.00000;AN_XX=77854;nhomalt_XX=0;AC_XY=4;AF_XY=5.37793e-05;AN_XY=74378;nhomalt_XY=0;nhomalt=0;AC_afr_XX=0;AF_afr_XX=0.00000;AN_afr_XX=22160;nhomalt_afr_XX=0;AC_afr_XY=0;AF_afr_XY=0.00000;AN_afr_XY=19310;nhomalt_afr_XY=0;AC_afr=0;AF_afr=0.00000;AN_afr=41470;nhomalt_afr=0;AC_ami_XX=0;AF_ami_XX=0.00000;AN_ami_XX=470;nhomalt_ami_XX=0;AC_ami_XY=0;AF_ami_XY=0.00000;AN_ami_XY=442;nhomalt_ami_XY=0;AC_ami=0;AF_ami=0.00000;AN_ami=912;nhomalt_ami=0;AC_amr_XX=0;AF_amr_XX=0.00000;AN_amr_XX=6790;nhomalt_amr_XX=0;AC_amr_XY=4;AF_amr_XY=0.000471032;AN_amr_XY=8492;nhomalt_amr_XY=0;AC_amr=4;AF_amr=0.000261746;AN_amr=15282;nhomalt_amr=0;AC_asj_XX=0;AF_asj_XX=0.00000;AN_asj_XX=1864;nhomalt_asj_XX=0;AC_asj_XY=0;AF_asj_XY=0.00000;AN_asj_XY=1604;nhomalt_asj_XY=0;AC_asj=0;AF_asj=0.00000;AN_asj=3468;nhomalt_asj=0;AC_eas_XX=0;AF_eas_XX=0.00000;AN_eas_XX=22

In [103]:
fields = line.strip().split('\t')


combined_row.update({"Chromosome": fields[0]})

combined_row.update({"Position": fields[1]})

combined_row.update({"rsIDs": fields[2]})

combined_row.update({"Reference": fields[3]})

combined_row.update({"Alternate": fields[4]})

last_field = fields[-1]

a = last_field.find("AC=")

a += len("AC=")

b = last_field[a:].find(";")

combined_row.update({"Allele Count": last_field[a:a+b]})

a = last_field.find("AN=")

a += len("AN=")

b = last_field[a:].find(";")

combined_row.update({"Allele Number": last_field[a:a+b]})

a = last_field.find("AF=")

a += len("AF=")

b = last_field[a:].find(";")

combined_row.update({"Allele Frequency": last_field[a:a+b]})



a = last_field.find("AC_ami=")

a += (len("AC_ami="))

b = last_field[a:].find(";")

combined_row.update({"Allele Count Amish": last_field[a:a+b]})

a = last_field.find("AN_ami=")

a += (len("AN_ami="))

b = last_field[a:].find(";")

combined_row.update({"Allele Number Amish": last_field[a:a+b]})

a = last_field.find("AF_ami=")

a += (len("AF_ami="))

b = last_field[a:].find(";")

combined_row.update({"Allele Frequency Amish": last_field[a:a+b]})


a = last_field.find("AC_amr=")

a += (len("AC_amr="))

b = last_field[a:].find(";")

combined_row.update({"Allele Count Admixed American": last_field[a:a+b]})

a = last_field.find("AN_amr=")

a += (len("AN_amr="))

b = last_field[a:].find(";")

combined_row.update({"Allele Number Admixed American": last_field[a:a+b]})

a = last_field.find("AF_amr=")

a += (len("AF_amr="))

b = last_field[a:].find(";")

combined_row.update({"Allele Frequency Admixed American": last_field[a:a+b]})




a = last_field.find("AC_asj=")

a += (len("AC_asj="))

b = last_field[a:].find(";")

combined_row.update({"Allele Count Ashkenazi Jewish": last_field[a:a+b]})

a = last_field.find("AN_asj=")

a += (len("AN_asj="))

b = last_field[a:].find(";")

combined_row.update({"Allele Number Ashkenazi Jewish": last_field[a:a+b]})

a = last_field.find("AF_asj=")

a += (len("AF_asj="))

b = last_field[a:].find(";")

combined_row.update({"Allele Frequency Ashkenazi Jewish": last_field[a:a+b]})



a = last_field.find("AC_eas=")

a += (len("AC_eas="))

b = last_field[a:].find(";")

combined_row.update({"Allele Count East Asian": last_field[a:a+b]})

a = last_field.find("AN_eas=")

a += (len("AN_eas="))

b = last_field[a:].find(";")

combined_row.update({"Allele Number East Asian": last_field[a:a+b]})

a = last_field.find("AF_eas=")

a += (len("AF_eas="))

b = last_field[a:].find(";")

combined_row.update({"Allele Frequency East Asian": last_field[a:a+b]})


a = last_field.find("AC_fin=")

a += (len("AC_fin="))

b = last_field[a:].find(";")

combined_row.update({"Allele Count European (Finnish)": last_field[a:a+b]})

a = last_field.find("AN_fin=")

a += (len("AN_fin="))

b = last_field[a:].find(";")

combined_row.update({"Allele Number European (Finnish)": last_field[a:a+b]})

a = last_field.find("AF_fin=")

a += (len("AF_fin="))

b = last_field[a:].find(";")

combined_row.update({"Allele Frequency European (Finnish)": last_field[a:a+b]})


a = last_field.find("AC_nfe=")

a += (len("AC_nfe="))

b = last_field[a:].find(";")

combined_row.update({"Allele Count European (non-Finnish)": last_field[a:a+b]})

a = last_field.find("AN_nfe=")

a += (len("AN_nfe="))

b = last_field[a:].find(";")

combined_row.update({"Allele Number European (non-Finnish)": last_field[a:a+b]})

a = last_field.find("AF_nfe=")

a += (len("AF_nfe="))

b = last_field[a:].find(";")

combined_row.update({"Allele Frequency European (non-Finnish)": last_field[a:a+b]})


a = last_field.find("AC_sas=")

a += (len("AC_sas="))

b = last_field[a:].find(";")

combined_row.update({"Allele Count South Asian": last_field[a:a+b]})

a = last_field.find("AN_sas=")

a += (len("AN_sas="))

b = last_field[a:].find(";")

combined_row.update({"Allele Number South Asian": last_field[a:a+b]})

a = last_field.find("AF_sas=")

a += (len("AF_sas="))

b = last_field[a:].find(";")

combined_row.update({"Allele Frequency South Asian": last_field[a:a+b]})


a = last_field.find("AC_remaining=")

a += (len("AC_remaining="))

b = last_field[a:].find(";")

combined_row.update({"Allele Count Remaining": last_field[a:a+b]})

a = last_field.find("AN_remaining=")

a += (len("AN_remaining="))

b = last_field[a:].find(";")

combined_row.update({"Allele Number Remaining": last_field[a:a+b]})

a = last_field.find("AF_remaining=")

a += (len("AF_remaining="))

b = last_field[a:].find(";")

combined_row.update({"Allele Frequency Remaining": last_field[a:a+b]})


a = last_field.find("AC_afr=")

a += (len("AC_afr="))

b = last_field[a:].find(";")

combined_row.update({"Allele Count African/African American": last_field[a:a+b]})

a = last_field.find("AN_afr=")

a += (len("AN_afr="))

b = last_field[a:].find(";")

combined_row.update({"Allele Number African/African American": last_field[a:a+b]})

a = last_field.find("AF_afr=")

a += (len("AF_afr="))

b = last_field[a:].find(";")

combined_row.update({"Allele Frequency African/African American": last_field[a:a+b]})

In [44]:
last_field

['AC=2',
 'AN=152308',
 'AF=1.31313e-05',
 'grpmax=afr',
 'fafmax_faf95_max=7.99000e-06',
 'fafmax_faf95_max_gen_anc=afr',
 'AC_XX=0',
 'AF_XX=0.00000',
 'AN_XX=77890',
 'nhomalt_XX=0',
 'AC_XY=2',
 'AF_XY=2.68752e-05',
 'AN_XY=74418',
 'nhomalt_XY=0',
 'nhomalt=0',
 'AC_afr_XX=0',
 'AF_afr_XX=0.00000',
 'AN_afr_XX=22168',
 'nhomalt_afr_XX=0',
 'AC_afr_XY=2',
 'AF_afr_XY=0.000103541',
 'AN_afr_XY=19316',
 'nhomalt_afr_XY=0',
 'AC_afr=2',
 'AF_afr=4.82114e-05',
 'AN_afr=41484',
 'nhomalt_afr=0',
 'AC_ami_XX=0',
 'AF_ami_XX=0.00000',
 'AN_ami_XX=470',
 'nhomalt_ami_XX=0',
 'AC_ami_XY=0',
 'AF_ami_XY=0.00000',
 'AN_ami_XY=442',
 'nhomalt_ami_XY=0',
 'AC_ami=0',
 'AF_ami=0.00000',
 'AN_ami=912',
 'nhomalt_ami=0',
 'AC_amr_XX=0',
 'AF_amr_XX=0.00000',
 'AN_amr_XX=6792',
 'nhomalt_amr_XX=0',
 'AC_amr_XY=0',
 'AF_amr_XY=0.00000',
 'AN_amr_XY=8502',
 'nhomalt_amr_XY=0',
 'AC_amr=0',
 'AF_amr=0.00000',
 'AN_amr=15294',
 'nhomalt_amr=0',
 'AC_asj_XX=0',
 'AF_asj_XX=0.00000',
 'AN_asj_XX=1868',
 

In [58]:
a = fields[-1].find("AC=")

a += len("AC=")

b = fields[-1][a:].find(";")

fields[-1][a:a+b]

'2'

In [104]:
combined_row

{'chromo': 'chr21',
 'strand': '-',
 'start': 5124755,
 'end': 5124875,
 'genename': 'CH507-9B2.3,CH507-9B2.4',
 'Major_Exon_Size': 120,
 'count': 202,
 'alt5': 7,
 'constitLevel': 0.967,
 'alt5usageLevel': 0.033,
 'alt5singleCount': 7,
 'alt5singleLoc': 5124674,
 'Alt_Exon_Size': 201,
 '5.ss.Con': 'chr21:5124749-5124758',
 '5.ss.Seq': 'CCTGTGCGT',
 'MaxENT': 4.27,
 '5.ss.Alt': 'chr21:5124668-5124677',
 '5.ss.Alt.Seq': 'GTGGTCAGT',
 'ALTMaxENT': 4.85,
 '?MaxEnt': -0.58,
 'Upstream_intronSize': 149,
 'Downstream_intronSize': 2952,
 'Upperstream_intron_gcpct': 0.591241,
 'Exon_gcpct': 0.53913,
 'Downstream_intron_gcpct': 0.54,
 'Alt5Exon_gcpct': 0.545918,
 'Alt5Downstream_intron_gcpct': 0.58,
 'Chromosome': 'chr21',
 'Position': '10000003',
 'Allele Count': '2',
 'Allele Number': '152308',
 'Allele Frequency': '1.31313e-05',
 'rsIDs': 'rs1473816357',
 'Reference': 'A',
 'Alternate': 'T',
 'Allele Count Amish': '0',
 'Allele Number Amish': '912',
 'Allele Frequency Amish': '0.00000',
 'Al

In [90]:
a = last_field.find("AC_ami=")

a += (len("AC_ami="))

b = last_field[a:].find(";")

last_field[a:a+b]



'0'

In [83]:
last_field

'AC=2;AN=152308;AF=1.31313e-05;grpmax=afr;fafmax_faf95_max=7.99000e-06;fafmax_faf95_max_gen_anc=afr;AC_XX=0;AF_XX=0.00000;AN_XX=77890;nhomalt_XX=0;AC_XY=2;AF_XY=2.68752e-05;AN_XY=74418;nhomalt_XY=0;nhomalt=0;AC_afr_XX=0;AF_afr_XX=0.00000;AN_afr_XX=22168;nhomalt_afr_XX=0;AC_afr_XY=2;AF_afr_XY=0.000103541;AN_afr_XY=19316;nhomalt_afr_XY=0;AC_afr=2;AF_afr=4.82114e-05;AN_afr=41484;nhomalt_afr=0;AC_ami_XX=0;AF_ami_XX=0.00000;AN_ami_XX=470;nhomalt_ami_XX=0;AC_ami_XY=0;AF_ami_XY=0.00000;AN_ami_XY=442;nhomalt_ami_XY=0;AC_ami=0;AF_ami=0.00000;AN_ami=912;nhomalt_ami=0;AC_amr_XX=0;AF_amr_XX=0.00000;AN_amr_XX=6792;nhomalt_amr_XX=0;AC_amr_XY=0;AF_amr_XY=0.00000;AN_amr_XY=8502;nhomalt_amr_XY=0;AC_amr=0;AF_amr=0.00000;AN_amr=15294;nhomalt_amr=0;AC_asj_XX=0;AF_asj_XX=0.00000;AN_asj_XX=1868;nhomalt_asj_XX=0;AC_asj_XY=0;AF_asj_XY=0.00000;AN_asj_XY=1604;nhomalt_asj_XY=0;AC_asj=0;AF_asj=0.00000;AN_asj=3472;nhomalt_asj=0;AC_eas_XX=0;AF_eas_XX=0.00000;AN_eas_XX=2282;nhomalt_eas_XX=0;AC_eas_XY=0;AF_eas_XY=0.0